 # Algorithm Verification Notebook
     
This notebook is used to test our custom optimization algorithms against the trusted implementations 
in the SciPy library. We will test the following algorithms:
     
    1.  Bisection Method
    2.  Golden Section Search
    3.  Newton-Raphson Method
    4.  Secant Method
     
Each test will define a sample function, run our implementation, run the SciPy implementation, and
      print the results for comparison.

In [30]:
import numpy as np
import sys, os
sys.path.append(os.path.abspath(".."))
from scipy.optimize import bisect, golden, newton, minimize_scalar

# Import the written algorithms
from optimization_algorithms.bisection import bisection as my_bisection
from optimization_algorithms.golden_search import golden_search as my_golden_search
from optimization_algorithms.newton_raphson import newton_raphson as my_newton_raphson
from optimization_algorithms.secant import secant as my_secant

# --- 1. Bisection Method Verification ---

bisection_tests = [
    {"func": lambda x: x**2 - 1, "a": 0, "b": 2, "true_root": 1},
    {"func": lambda x: x**3 - 8, "a": 1, "b": 3, "true_root": 2},
    {"func": lambda x: x**5 - 3125, "a": 4, "b": 6, "true_root": 5},
]

print("--- 1. Bisection Method ---")
for i, test in enumerate(bisection_tests, start=1):
    try:
        f = test["func"]
        my_res = my_bisection(f, test["a"], test["b"])
        scipy_res = bisect(f, test["a"], test["b"])
        print(f"Test {i}: Expected root ~ {test['true_root']}")
        print(f"  My implementation: {my_res['root']}")
        print(f"  SciPy result:      {scipy_res}")
    except Exception as e:
        print(f"  Error in Bisection Test {i}: {e}")
print("-" * 60)


# --- 2. Secant Method Verification ---


secant_tests = [
    {"func": lambda x: (x * (x**2 + x - 1)) / (x + 1),
     "x0": 0.2, "x1": 0.7, "expected_root": 0.6180339887},

    {"func": lambda x: (x - 1) * (x - 2) * (x - 3),
     "x0": 1.5, "x1": 2.5, "expected_root": 2.0},

    {"func": lambda x: np.cos(x) - x,
     "x0": 0.5, "x1": 1.0, "expected_root": 0.7390851332},
]

print("\n--- 2. Secant Method ---")
for i, test in enumerate(secant_tests, start=1):
    try:
        f = test["func"]
        my_res = my_secant(f, test["x0"], test["x1"])
        scipy_res = newton(f, x0=test["x0"], x1=test["x1"])
        print(f"Test {i}: Expected root ~ {test['expected_root']}")
        print(f"  My implementation: {my_res['root']}")
        print(f"  SciPy result:      {scipy_res}")
    except Exception as e:
        print(f"  Error in Secant Test {i}: {e}")
print("-" * 60)



# --- 3. Newton-Raphson Method Verification ---

newton_tests = [
    {"func": lambda x: x**3 - 2*x - 5, "df": lambda x: 3*x**2 - 2, "x0": 2.0, "true_root": 2.094},
    {"func": lambda x: x**3 - x - 2, "df": lambda x: 3*x**2 - 1, "x0": 1.0, "true_root": 1.521},
    {"func": lambda x: np.cos(x) - x, "df": lambda x: -np.sin(x) - 1, "x0": 0.5, "true_root": 0.739},
]

print("\n--- 3. Newton-Raphson Method ---")
for i, test in enumerate(newton_tests, start=1):
    try:
        f, df = test["func"], test["df"]
        my_res = my_newton_raphson(f, df, x0=test["x0"])
        scipy_res = newton(f, x0=test["x0"], fprime=df)
        print(f"Test {i}: Expected root ~ {test.get('true_root')}")
        print(f"  My implementation: {my_res['root']}")
        print(f"  SciPy result:      {scipy_res}")
    except Exception as e:
        print(f"  Error in Newton-Raphson Test {i}: {e}")
print("-" * 60)


# --- 4. Golden Section Search Verification (corrected) ---

golden_tests = [
    # Test 1: Quadratic minimum
    {"func": lambda x: 3*x**2 + 20*x - 1, "bracket": (-7, 7), "expected_min": -3.3333333333},

    # Test 2: Sinusoidal minimum near -π/4
    {"func": lambda x: np.sin(3*x) + np.cos(3*x), "bracket": (-1.5, -0.5), "expected_min": -0.785398},

    # Test 3: Absolute value function (non-negative)
    {"func": lambda x: np.abs(np.sin(x) * np.log(x + 2.5)), "bracket": (-2.4, 2), "expected_min": 0.0},
]

print("\n--- 4. Golden Section Search ---")
for i, test in enumerate(golden_tests, start=1):
    try:
        f = test["func"]
        my_res = my_golden_search(f, a=test["bracket"][0], b=np.mean(test["bracket"]), c=test["bracket"][1])
        
        # Use bounded minimization for SciPy to respect the boundaries
        if i == len(golden_tests):  # only last test uses bounded
            scipy_res = minimize_scalar(f, bounds=(test["bracket"][0], test["bracket"][1]), method='bounded')
            scipy_x = scipy_res.x
        else:  # use golden for the others if you want
            from scipy.optimize import golden
            scipy_x = golden(f, brack=test["bracket"])
            
        print(f"Test {i}: Expected minimum x ~ {test['expected_min']}")
        print(f"  My implementation (x): {my_res['minimum']}")
        print(f"  SciPy result (x):      {scipy_x}")
        print(f"  f(My x) = {f(my_res['minimum'])}, f(SciPy x) = {f(scipy_x)}")
    except Exception as e:
        print(f"  Error in Golden Section Search Test {i}: {e}")
print("-" * 60)

print("\n========== All Tests Completed ==========\n")


--- 1. Bisection Method ---
Test 1: Expected root ~ 1
  My implementation: 1.0
  SciPy result:      1.0
Test 2: Expected root ~ 2
  My implementation: 2.0
  SciPy result:      2.0
Test 3: Expected root ~ 5
  My implementation: 5.0
  SciPy result:      5.0
------------------------------------------------------------

--- 2. Secant Method ---
Test 1: Expected root ~ 0.6180339887
  My implementation: 0.6180339887498948
  SciPy result:      0.618033988749895
Test 2: Expected root ~ 2.0
  My implementation: 2.0
  SciPy result:      2.0
Test 3: Expected root ~ 0.7390851332
  My implementation: 0.7390851332151607
  SciPy result:      0.7390851332151605
------------------------------------------------------------

--- 3. Newton-Raphson Method ---
Test 1: Expected root ~ 2.094
  My implementation: 2.0945514815423265
  SciPy result:      2.0945514815423265
Test 2: Expected root ~ 1.521
  My implementation: 1.5213797068045676
  SciPy result:      1.5213797068045676
Test 3: Expected root ~ 0.739
 